In [8]:
import pyspark
import pprint 
from numpy import array
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel



pp = pprint.PrettyPrinter(indent=2)

def getRDD(fileName):
    rdd = sc.textFile(fileName)
    cols = rdd.map(lambda line: line.split(','))
    
    return cols

def getSpeciesCount(rdd):
    pp.pprint(rdd[0])
    
def countSpecies(rdd):
    pairs = rdd.map(lambda a: (a,1))
    counts = pairs.reduceByKey(lambda x,y: x+y)
    print(counts.takeOrdered(10, key=lambda p: p[-1]))
    
def getYears(rdd):
    dates = [x[6].split("-")[0] for x in rdd.collect() if len(x[6].split("-"))== 3 ]
    return list(set(dates))

def cluster(rdd):
    pass
    
if __name__ == "__main__":
    cols = getRDD('/home/WUR/steen176/Downloads/butterflies.csv')
    
    species = cols.map(lambda x: x[0])
    birds = cols.filter(lambda x: True if x[2] == u"1" else False)
    butterflies = cols.filter(lambda x: True if x[2] == u"4" else False)
    
    countSpecies(species)
    #year_list = getYears(cols)
    #print(year_list)
    
    




[(u'species', 1), (u'Issoria lathonia', 36), (u'Asio flammeus', 320), (u'Aricia agestis', 432), (u'Thymelicus sylvestris', 617), (u'Streptopelia turtur', 669), (u'Favonius quercus', 744), (u'Dendrocopos medius', 925), (u'Hesperia comma', 977), (u'Melitaea athalia', 1066)]


In [114]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.mllib.linalg import DenseVector
from pyspark.mllib.linalg import SparseVector
from numpy import array
import re
import numpy
from math import sqrt

NUM_CLUSTER = 4

data=sc.textFile("/home/WUR/steen176/Downloads/test.dat")

months = {1:"winter",
          2:'winter',
          3:"winter",
          4:"spring",
          5:'spring',
          6:"spring",
          7:'summer',
          8:"summer",
          9:"summer",
          10:'autum',
          11:'autum',
          12:"autum"}

def parseline(line):
    line = line.split(',')
    longti = float(line[8])
    latti = float(line[9])
    s_vec = SparseVector(2, [[0, longti], [1, latti]])
    return s_vec 


def getVar(line):
    line = line.split(',')
    return float(line[8]) - float(line[9])

parsedData = data.map(lambda line: parseline(line))
var = data.map(lambda line: getVar(line))
var = var.toLocalIterator()
var = [x for x in content]

num_clusters = (len(var)-1)* sum(var) *-1

clusters = KMeans.train(parsedData, NUM_CLUSTER, maxIterations=20, runs=100, initializationMode="random")

def error(point):
    center = clusters.centers[clusters.predict(point)]
    denseCenter = DenseVector(numpy.ndarray.tolist(center))
    return sqrt(sum([x**2 for x in (DenseVector(point.toArray()) - denseCenter)]))

WSSSE = parsedData.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print(WSSSE)
print(clusters.predict(array([5.9737, 52.4066])))


108993.352881
1.15190214839
1
